In [1]:
import os
import sys

import copy
import numpy as onp
from astropy.cosmology import Planck18

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd())))
sys.path.append(SCRIPT_DIR)


In [2]:
import GWfish.fisherGlobals as glob

## COMPLETE EXAMPLE: GW170817

###  We use the positions of existing LVC detectors

In [3]:
alldetectors = copy.deepcopy(glob.detectors)
print('All available detectors are: '+str(list(alldetectors.keys())))

# select only LIGO and Virgo
LVdetectors = {det:alldetectors[det] for det in ['L1', 'H1', 'Virgo']}
print('Using detectors '+str(list(LVdetectors.keys())))


All available detectors are: ['L1', 'H1', 'Virgo', 'Kagra', 'ETS']
Using detectors ['L1', 'H1', 'Virgo']


In [4]:
# We use the O2 psds
LVdetectors['L1']['psd_path'] = os.path.join(glob.detPath, 'LVK_O1O2O3', '2017-08-06_DCH_C02_L1_O2_Sensitivity_strain_asd.txt')
LVdetectors['H1']['psd_path'] = os.path.join(glob.detPath, 'LVK_O1O2O3', '2017-06-10_DCH_C02_H1_O2_Sensitivity_strain_asd.txt')
LVdetectors['Virgo']['psd_path'] = os.path.join(glob.detPath, 'LVK_O1O2O3', 'Hrec_hoft_V1O2Repro2A_16384Hz.txt')


In [5]:
from GWfish.waveforms import IMRPhenomD_NRTidalv2
from GWfish.signal import GWSignal
from GWfish.network import DetNet
from fisherTools import CovMatr, compute_localization_region, check_covariance, fixParams


### Initialise the signals and then the network

In [6]:

myLVSignals = {}

for d in LVdetectors.keys():

    myLVSignals[d] = GWSignal(IMRPhenomD_NRTidalv2(), 
                psd_path=LVdetectors[d]['psd_path'],
                detector_shape = LVdetectors[d]['shape'],
                det_lat= LVdetectors[d]['lat'],
                det_long=LVdetectors[d]['long'],
                det_xax=LVdetectors[d]['xax'], 
                verbose=True,
                useEarthMotion = False,
                fmin=10.,
                IntTablePath=None) 
        
myLVNet = DetNet(myLVSignals)      

Using ASD from file /Users/francesco.iacovelli/Desktop/PhD/Research/2021-09_paper_BNS_massfun_MCMC1/GWfish/psds/LVK_O1O2O3/2017-08-06_DCH_C02_L1_O2_Sensitivity_strain_asd.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8
Using ASD from file /Users/francesco.iacovelli/Desktop/PhD/Research/2021-09_paper_BNS_massfun_MCMC1/GWfish/psds/LVK_O1O2O3/2017-06-10_DCH_C02_H1_O2_Sensitivity_strain_asd.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8
Using ASD from file /Users/francesco.iacovelli/Desktop/PhD/Research/2021-09_paper_BNS_massfun_MCMC1/GWfish/psds/LVK_O1O2O3/Hrec_hoft_V1O2Repro2A_16384Hz.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8


In [7]:
from fisherUtils import GPSt_to_LMST

# Median values of the posterior samples for all the parameters, 
# except psi and the coalescence phase that are set to 0

z = onp.array([0.00980])
tGPS = onp.array([1187008882.4])

GW170817 = {'Mc':onp.array([1.1859])*(1.+z), 
            'dL':Planck18.luminosity_distance(z).value/1000., 
            'theta':onp.array([onp.pi/2. + 0.4080839999999999]), 
            'phi':onp.array([3.4461599999999994]),
            'iota':onp.array([2.545065595974997]), 
            'psi':onp.array([0.]),
            'tcoal':GPSt_to_LMST(tGPS, lat=0., long=0.), # GMST is LMST computed at long = 0° 
            'eta':onp.array([0.24786618323504223]), 
            'Phicoal':onp.array([0.]), 
            'chi1z':onp.array([0.005136138323169717]), 
            'chi2z':onp.array([0.003235146993487445]), 
            'Lambda1':onp.array([368.17802383555687]), 
            'Lambda2':onp.array([586.5487031450857])
           }

print('Parameters for GW170817 are:')
GW170817

Parameters for GW170817 are:


{'Mc': array([1.19752182]),
 'dL': array([0.04374755]),
 'theta': array([1.97888033]),
 'phi': array([3.44616]),
 'iota': array([2.5450656]),
 'psi': array([0.]),
 'tcoal': DeviceArray([0.43432288], dtype=float64),
 'eta': array([0.24786618]),
 'Phicoal': array([0.]),
 'chi1z': array([0.00513614]),
 'chi2z': array([0.00323515]),
 'Lambda1': array([368.17802384]),
 'Lambda2': array([586.54870315])}

### Compute the expected SNR (true was 33)

In [8]:
SNR = myLVNet.SNR(GW170817)
print('SNR for GW170817 is %.2f to compare with 33'%SNR[0])

SNR for GW170817 is 33.16 to compare with 33


### Compute the total Fisher matrix

In [9]:
totF = myLVNet.FisherMatr(GW170817)
print('The computed Fisher matrix has shape %s'%str(totF.shape))

Computing Fisher for L1...
Computing derivatives...
Computing Fisher for H1...
Computing derivatives...
Computing Fisher for Virgo...
Computing derivatives...
Computing total Fisher ...
Done.
The computed Fisher matrix has shape (13, 13, 1)


In [10]:
# Check e.g. that the (dL,dL) element corresponds to (SNR/dL)^2
ParNums = IMRPhenomD_NRTidalv2().ParNums
dL_Num = ParNums['dL']
print('The relative difference is %.2e !'%((1 - totF[ParNums['dL'],ParNums['dL'],:]/(SNR/GW170817['dL'])**2)[0]))


The relative difference is 0.00e+00 !


### Compute the covariance and perform some checks

In [11]:
totCov, _ = CovMatr(totF, GW170817)


 Inversion error with method cho: min=1.25732421875, max=1.25732421875, mean=1.25732421875, std=0.0 
Method cho not possible on 0 non-positive definite matrices, svd was used in those cases. 


In [12]:
_ = check_covariance(totF, totCov)


Inversion errors: [1.4755030262032268473]
diagonal-1 = [array([ 7.13090031e-10,  3.92552124e-09,  1.39756511e-12, -1.73710872e-15,
        1.52885517e-13,  5.01339205e-13, -7.25974836e-13, -2.94711811e-10,
        1.13512755e-10,  6.48310561e-08, -4.56286671e-08,  4.44717874e-12,
        1.80762072e-10], dtype=float128)]
Max off diagonal: [0.04788970947265625]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False],
       [False,  True, False, False, False, False, False, False, False,
         True,  True, False, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False, False],
       [False, False, False, False,  True, False, False, False, False,
        False, False, False, False],
       [ True, False

### Now try to eliminate the row corresponding to $\delta\tilde{\Lambda}$, and see that the inversion error lowers

In [13]:
ParNums = IMRPhenomD_NRTidalv2().ParNums

newFish, newPars = fixParams(totF, ParNums, ['deltaLambda'])

print('Now the Fisher matrix has shape %s'%str(newFish.shape))

newCov, _ = CovMatr(newFish, GW170817)

_ = check_covariance(newFish, newCov)


Now the Fisher matrix has shape (12, 12, 1)
 Inversion error with method cho: min=0.00048416515346616506577, max=0.00048416515346616506577, mean=0.00048416515346616506577, std=0.0 
Method cho not possible on 0 non-positive definite matrices, svd was used in those cases. 
Inversion errors: [0.00049634754712455514403]
diagonal-1 = [array([-4.53178883e-10,  4.24105706e-09,  1.25314906e-12, -3.35988832e-15,
        2.09935910e-13,  7.83653741e-14,  9.42618986e-12, -1.16235514e-10,
        7.72211600e-11, -2.24066593e-07,  1.20702719e-07,  3.92778518e-12],
      dtype=float128)]
Max off diagonal: [8.459925834358728025e-05]

mask: where F*S(off-diagonal)>1e-10 (--> problematic if True off diagonal)
[array([[ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
         True,  True, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, Fa

### Finally compute the localisation region

In [14]:
skyArea = compute_localization_region(newCov, newPars, GW170817['theta'])
print('The estimated sky location is %.1f deg^2, to compare with 16 deg^2'%skyArea)


The estimated sky location is 19.9 deg^2, to compare with 16 deg^2


## Now an example with ET and multiple events together

### Configure the interforometer's properties

In [15]:
# Configure ET and the PSD
ETdet = {'ET': copy.deepcopy(glob.detectors).pop('ETS') }
print(ETdet)
ETdet['ET']['psd_path'] = os.path.join(glob.detPath, 'pycbc_public', 'et_d.txt')

{'ET': {'lat': 40.516666666666666, 'long': 9.416666666666666, 'xax': 0.0, 'shape': 'T'}}


### Build the DetNet object

In [16]:
from GWfish.waveforms import TaylorF2_RestrictedPN
from GWfish.signal import GWSignal
from GWfish.network import DetNet

In [17]:
mySignalsET = {}

for d in ETdet.keys():

    mySignalsET[d] = GWSignal(TaylorF2_RestrictedPN(use_3p5PN_SpinHO=True, is_tidal=True), 
                psd_path= ETdet[d]['psd_path'],
                detector_shape = ETdet[d]['shape'],
                det_lat= ETdet[d]['lat'],
                det_long=ETdet[d]['long'],
                det_xax=ETdet[d]['xax'], 
                verbose=True,
                useEarthMotion = True,
                fmin=2.,
                IntTablePath=None) 

myNet = DetNet(mySignalsET) 

Using ASD from file /Users/francesco.iacovelli/Desktop/PhD/Research/2021-09_paper_BNS_massfun_MCMC1/GWfish/psds/pycbc_public/et_d.txt 
Initializing jax...
Jax local device count: 8
Jax  device count: 8


### Sample some BNS-like events

In [18]:
nevents=100

zs = onp.random.uniform(1e-2, 3., nevents)

dLs = Planck18.luminosity_distance(zs).value/1000

Mcs = onp.random.normal(loc=1.156, scale=0.056, size=nevents) 
   
events_rand = {'Mc': Mcs*(1.+zs), 
               'eta': onp.random.uniform(0.24, 0.25, nevents), 
               'dL': dLs, 
               'theta':onp.arccos(onp.random.uniform(-1., 1., nevents)), 
               'phi':onp.random.uniform(0., 2.*onp.pi, nevents), 
               'iota':onp.arccos(onp.random.uniform(-1., 1., nevents)), 
               'psi':onp.random.uniform(0., 2.*onp.pi, nevents), 
               'tcoal':onp.random.uniform(0., 1., nevents), 
               'Phicoal': onp.random.uniform(0., 2.*onp.pi, nevents),
               'chi1z':onp.random.uniform(-.05, .05, nevents), 
               'chi2z':onp.random.uniform(-.05, .05, nevents), 
               'Lambda1':onp.random.uniform(0., 2000., nevents), 
               'Lambda2':onp.random.uniform(0., 2000., nevents),
              }


### Then computing SNRs and Fisher matrices is as easy and fast as

In [19]:
%%time
snrs = myNet.SNR(events_rand)

CPU times: user 842 ms, sys: 101 ms, total: 942 ms
Wall time: 835 ms


In [20]:
%%time
totF = myNet.FisherMatr(events_rand)

Computing Fisher for ET...
Computing derivatives...
Filling matrix for arm 1...
Computing derivatives...
Filling matrix for arm 2...
Filling matrix for arm 3...
Computing total Fisher ...
Done.
CPU times: user 5min 23s, sys: 4min 22s, total: 9min 45s
Wall time: 3min 12s
